In [2]:
import numpy as np
import chipwhisperer as cw
import chipwhisperer.analyzer as cwa
from chipwhisperer.analyzer.attacks.models.aes.key_schedule import key_schedule_rounds
import os

## Intialize ChipWhisperer Analyzer Module

In [3]:
# Load data file
data = np.load("data/traces_1.npz", allow_pickle=True)
plaintexts = data['dut_io_data']
ciphertexts = data['dut_io_computed_data']
traces = data['wave']

# Create a dummy key of the same length
key_placeholder = bytearray(16)

# Create the CW project and append traces
# Make sure the 'data/traces_1' directory exists
if not os.path.exists("data/traces_1"):
    os.makedirs("data/traces_1")
    
proj = cw.create_project("data/traces_1/CW_305.cwp", overwrite=True)

print("Processing and appending traces to the project...")
for i, (wave, pt_raw, ct_raw) in enumerate(zip(traces, plaintexts, ciphertexts)):
    pt = pt_raw
    ct = ct_raw
    
    # Sanity check: ensure all blocks have the correct fixed length
    assert len(pt) == 16 and len(ct) == 16, \
           f"Trace {i}: Got pt/ct lengths {len(pt)}/{len(ct)}, expected {16}"
           
    proj.traces.append(cw.Trace(wave, pt, ct, key_placeholder))

print(f"Successfully processed {len(proj.traces)} traces.")
#proj.save()  # save the project for later use


Processing and appending traces to the project...
Successfully processed 2000 traces.


### Perform a CPA attack on the Last round

In [4]:
print("Instantiating and running CPA attack...")
# Set leakage model:
leak_model = cwa.leakage_models.last_round_state_diff
# Instantiate and run CPA
attack = cwa.cpa(proj, leak_model)
cb = cwa.get_jupyter_callback(attack)
best_guesses = attack.run(cb)

print("\n--- Attack Results ---")
print("Recovered key-byte guesses:", best_guesses)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
PGE=,177,117,87,2,52,217,8,116,19,93,135,218,48,94,88,172
0,D50.175,5E0.169,840.189,A70.209,430.202,250.201,200.170,BC0.138,C00.135,240.212,830.198,880.181,1D0.188,150.156,880.169,000.151
1,E80.088,EC0.100,1B0.091,840.089,8A0.099,CB0.096,680.090,E00.095,970.097,460.101,3B0.092,600.090,290.089,730.102,3F0.094,180.107
2,4A0.086,240.099,460.088,CB0.081,BD0.089,530.093,710.085,FE0.091,AC0.090,F70.090,760.089,CE0.089,E00.087,E20.098,5E0.086,B00.101
3,0B0.084,F90.090,7B0.087,5E0.081,5A0.086,F60.089,430.083,E70.083,EA0.081,550.087,670.086,2D0.086,EF0.087,1C0.096,C80.084,F30.093
4,0C0.084,1D0.087,760.082,2A0.081,010.084,520.089,AA0.081,3D0.082,9B0.081,DD0.086,EA0.086,320.086,0B0.083,A40.094,670.083,0C0.088



--- Attack Results ---
Recovered key-byte guesses: Subkey KGuess Correlation
  00    0xD5    0.17502
  01    0x5E    0.16914
  02    0x84    0.18881
  03    0xA7    0.20878
  04    0x43    0.20177
  05    0x25    0.20134
  06    0x20    0.17032
  07    0xBC    0.13765
  08    0xC0    0.13475
  09    0x24    0.21179
  10    0x83    0.19846
  11    0x88    0.18064
  12    0x1D    0.18807
  13    0x15    0.15575
  14    0x88    0.16866
  15    0x00    0.15085



### Perform reverse schedule operation to recover the real Key

In [5]:
recv_lastroundkey = [kguess[0][0] for kguess in best_guesses.find_maximums()]
recv_key = key_schedule_rounds(recv_lastroundkey, 10, 0)
for subkey in recv_key:
    print(hex(subkey), end="")

0x100xa50x880x690xd70x4b0xe50xa30x740xcf0x860x7c0xfb0x470x380x59